In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import nltk
from nltk.corpus import stopwords

In [2]:
USER_AGENT = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
STOPWORDS = set(stopwords.words("portuguese"))

In [3]:
sites = pd.read_json('./Links/Sites.json')
stores = sites.columns
sites

,Gregory,Damyller,Dassi,Excluiva,CutiCutiBaby,BluK,Villevie,Achados96,DWZ,BellaSeda
negative,[https://www.gregory.com.br/institucional/sobr...,[https://www.damyller.com.br/boho-denim?O=Orde...,"[https://www.boutiquedassi.com.br/moda-praia, ...",[https://www.lojasexclusiva.com.br/shorts/Shor...,[https://www.cuticutibaby.com.br/vestido-gola-...,[https://www.bluk.com.br/vestido-longo-crepe-a...,[https://www.lojavillevie.com.br/kaftan-listra...,[https://www.achados96.com.br/acessorios/cinto...,[https://www.dwz.com.br/saia-jeans-barra-desfi...,[https://www.bellaseda.online/produto/289491/c...
positive,[https://www.gregory.com.br/blusa-cachecouer-m...,[https://www.damyller.com.br/blusa-decote-vaza...,[https://www.boutiquedassi.com.br/BLUSA-MANGA-...,[https://www.lojasexclusiva.com.br/t-shirts/T-...,[https://www.cuticutibaby.com.br/t-shirt-stree...,[https://www.bluk.com.br/bata-visco-linho-com-...,[https://www.lojavillevie.com.br/top-blusa/man...,[https://www.achados96.com.br/todos/t-shirt-ov...,[https://www.dwz.com.br/blusa-manga-longa-tule...,[https://www.bellaseda.online/produto/254694/b...


In [4]:
data = []
for store in stores:
    for i in range(0,2):
        for url in sites[store][i]:
            data.append([url, i])
df = pd.DataFrame(data, columns = ['url', 'label'])
df.head()

,url,label
0,https://www.gregory.com.br/institucional/sobre,0
1,https://www.gregory.com.br/promocoes,0
2,https://www.gregory.com.br/calca-cigarrete-de-...,0
3,https://www.gregory.com.br/institucional/troca...,0
4,https://www.gregory.com.br/colar-curto-pedrasc...,0


In [5]:
def clean(text):
    d = re.sub("[^a-zA-Z]", " ", text)
    words = d.lower().split()
    words = [w for w in words if w not in STOPWORDS]
    words = sorted(list(set(words)))
    return words

In [40]:
def content_page(page):
    try:
        content = open(f"./pages/{page}")
        s = BeautifulSoup(content, 'html.parser')
    except:
        req = requests.get(df.url[page], headers=USER_AGENT)
        content = req.text
        s = BeautifulSoup(content, 'html.parser')
    data = s.find_all(text=True)
    data = ''.join(data)
    text = data.replace('\n', ' ').replace('\r', '').strip()
    text = ' '.join(text.split())
    return clean(text)

In [42]:
dataset = []
for index, value in enumerate(df.values):
#     !cd pages/ && curl -o {index} {value[0]} && cd -
#     content = content_page(value[0])
    print(f"Lendo {index}: " + value[0])
    content = content_page(index)
    dataset.append([content, value[1]])
    time.sleep(1)
dataset = pd.DataFrame(dataset, columns=["content", "label"])

Lendo 0: https://www.gregory.com.br/institucional/sobre
Lendo 1: https://www.gregory.com.br/promocoes
Lendo 2: https://www.gregory.com.br/calca-cigarrete-de-sarjaverde-02-10-0783-024/p
Lendo 3: https://www.gregory.com.br/institucional/troca-loja-virtual
Lendo 4: https://www.gregory.com.br/colar-curto-pedrascoral-28-07-0089-070/p
Lendo 5: https://www.gregory.com.br/colar-curto-perolas-e-corrente-dourado-28-06-0069-134/p
Lendo 6: https://www.gregory.com.br/colar-curto-elos-resina-e-brilho-preto-28-06-0070-002/p
Lendo 7: https://www.gregory.com.br/novidades?O=OrderByReleaseDateDESC?O=OrderByReleaseDateDESC
Lendo 8: https://www.gregory.com.br/vestido-bicolor-preto-off-white-08-23-0125-196/p
Lendo 9: https://www.gregory.com.br/calca-cigarrete-xadrezmarrom-02-10-0758-043/p
Lendo 10: https://www.gregory.com.br/roupas-femininas/macacao?O=OrderByReleaseDateDESC?O=OrderByReleaseDateDESC
Lendo 11: https://www.gregory.com.br/macacao-decote-voff-white-22-02-0029-175/p
Lendo 12: https://www.gregory.

Lendo 108: https://www.cuticutibaby.com.br/bota-coturno-couro-suede-caramelo
Lendo 109: https://www.cuticutibaby.com.br/macacao-curto-las-bananas
Lendo 110: https://www.cuticutibaby.com.br/alpagarta-cuticuti-dark-jeans
Lendo 111: https://www.cuticutibaby.com.br/sapatilha-vegan-coelhinho
Lendo 112: https://www.cuticutibaby.com.br/t-shirt-street-mescla-listras
Lendo 113: https://www.cuticutibaby.com.br/moletom-street-bordo
Lendo 114: https://www.cuticutibaby.com.br/t-shirt-dont-grow-up
Lendo 115: https://www.cuticutibaby.com.br/t-shirt-raglan-long-ufofo
Lendo 116: https://www.cuticutibaby.com.br/regata-street-pinguinhos
Lendo 117: https://www.cuticutibaby.com.br/regata-street-flechas
Lendo 118: https://www.cuticutibaby.com.br/camisa-kids-amigos-no-bosque
Lendo 119: https://www.cuticutibaby.com.br/camisa-frufru-manga-essential
Lendo 120: https://www.cuticutibaby.com.br/blusao-conforto-plush-capuz
Lendo 121: https://www.cuticutibaby.com.br/camiseta-manga-longa-back
Lendo 122: https://www.b

Lendo 202: https://www.dwz.com.br/conteudo/sobre-a-dwz/sobre-voce
Lendo 203: https://www.dwz.com.br/bolsa-mini-190707004-p1142383?pp=/44.709/
Lendo 204: https://www.dwz.com.br/sandalia-verniz-rasteira-33207039-p1039229?pp=/44.709/
Lendo 205: https://www.dwz.com.br/bolsa-181007022-p1033386?pp=/44.709/
Lendo 206: https://www.dwz.com.br/macacao-longo-decote-v-alcas-finas-190905005-p1144256?pp=/44.769/
Lendo 207: https://www.dwz.com.br/blusa-manga-longa-tule-190704006-p1142134?pp=/44.709/
Lendo 208: https://www.dwz.com.br/blusa-manga-longa-tule-190704006-p1142134?pp=/44.573/
Lendo 209: https://www.dwz.com.br/blusa-lisa-180804017-p999384?pp=/44.709/
Lendo 210: https://www.dwz.com.br/blusa-alca-larga-de-linho-181104002-p1018756?pp=/44.634/
Lendo 211: https://www.dwz.com.br/blusa-alca-decote-v-190804004-p1142842?pp=/44.709/
Lendo 212: https://www.dwz.com.br/blusa-crepe-manga-curta-181204038-p985743?pp=/44.573/
Lendo 213: https://www.dwz.com.br/blusa-cropped-ombro-a-ombro-poa-190804015-p114352

In [31]:
dataset.to_csv(path_or_buf=f"./Dataframes/clf_content.csv", index=False)

AttributeError: 'list' object has no attribute 'to_csv'